In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle

from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.engine.topology import Layer
from keras.layers import Input, Dropout, Dense
from keras.layers import Embedding, GRU, LSTM, Bidirectional, TimeDistributed
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, Callback, LambdaCallback, TensorBoard
from keras.optimizers import SGD, Adam

%matplotlib inline

Using TensorFlow backend.


In [2]:
np.random.seed(3)
from tensorflow import set_random_seed
set_random_seed(24)

In [3]:
with open('data_final/reliable_final.pkl', 'rb') as infile:
    reliable = pickle.load(infile)

In [4]:
with open('data_final/unreliable_final.pkl', 'rb') as infile:
    unreliable = pickle.load(infile)

In [5]:
reliable['label'] = 0
unreliable['label'] = 1
data = reliable.append(unreliable).reset_index()

In [6]:
import gc
reliable = None
unreliable = None
gc.collect()

28

In [7]:
max_words = 30  # max num words processed for each sentence
max_sentences = 30  # max num sentences processed for each article 
max_vocab = 50000
embedding_dim = 100  # size of pretrained word vectors
attention_dim = 128  # num units in attention layer
GRU_dim = 64  # num units in GRU layer, but it is bidirectional so outputs double this number
batch_size = 128
test_size = 0.2

vector_dir = './embeddings'
vector_file = 'glove.6B.100d.txt'
model_dir = './model_output/glove_100'
tb_logs = './tb_logs/glove_100'

In [8]:
# Split data into train and test sets. Test set will not be used in any way until model is trained.
# The data splits created here will be saved 

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X_train, X_test, y_train, y_test = train_test_split(data['content'], data['label'], test_size=test_size,
                                                    random_state=77, stratify=data['label'])
data = None
gc.collect()

16

In [9]:
# Build vocab word index (dict) from training articles

from collections import Counter 
words = Counter()
for article in X_train:
    for sentence in article:
        sentence = [word.lower() for word in sentence]
        words.update(sentence)
    
#with open('data_final/words.pkl', 'wb') as outfile:
#    pickle.dump(words, outfile)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size,
                                                    random_state=77, stratify=y_train)

#with open('aws/data/X_train.pkl', 'wb') as outfile:
#    pickle.dump(X_train, outfile)
    
#with open('aws/data/X_val.pkl', 'wb') as outfile:
#    pickle.dump(X_val, outfile)

#with open('aws/data/X_test.pkl', 'wb') as outfile:
#    pickle.dump(X_test, outfile)

#with open('aws/data/y_train.pkl', 'wb') as outfile:
#    pickle.dump(y_train, outfile)
    
#with open('aws/data/y_val.pkl', 'wb') as outfile:
#    pickle.dump(y_val, outfile)

#with open('aws/data/y_test.pkl', 'wb') as outfile:
#    pickle.dump(y_test, outfile)   

An embedding matrix must be constructed so that the embedding layer can look up and use the vector associated with each word. As an intermediary step, the words are first replaced with their respective indices in the word index. The embedding matrix uses these same indices for the lookup.

Only the most common *n* words will be used from the word index. By pre-populating the data and embedding matrices with zeros, any words outside the *n*-word vocabulary or not found in the set of embedding vectors will be represented by a zero or vector of zeros. The GRU and attention layers support masking, which avoids having to process vectors of all zeros.

Trial runs to ensure that the network is working properly and to determine learning rate bounds (to be used with a cyclic learning rate approach) will be conducted with a small subset of the training data comprising 10,000 articles.

In [ ]:
for ix, (word, _) in enumerate(words.most_common(max_vocab)):
    word_index[word] = ix + 1 # The zero index is reserved for masking out-of-vocab words

In [ ]:
def create_data_matrix(data, max_sentences=max_sentences, max_words=max_words, max_vocab=max_vocab,
                      word_index=word_index):
    data_matrix = np.zeros((len(data), max_sentences, max_words), dtype='int32')
    for i, article in enumerate(data):
        for j, sentence in enumerate(article):
            if j == max_sentences:
                break
            k = 0
            for word in sentence:
                if k == max_words:
                    break
                ix = word_index.get(word.lower())
                if ix is not None and ix < max_vocab:
                    data_matrix[i, j, k] = ix
                k = k + 1
    return data_matrix  

In [ ]:
sample_X_train = X_train[X_train['label'] == 1].sample(4000, random_state=77).append(
    X_train[X_train['label'] == 1].sample(4000, random_state=77))
, sample_X_test, sample_y_train, sample_y_test = train_test_split(sample_data['content'],
                                                    sample_data['label'], test_size=test_size,
                                                    random_state=77, stratify=sample_data['label'])

In [ ]:
sample_train_matrix = create_data_matrix(sample_X_train)
sample_test_matrix = create_data_matrix(sample_X_test)

sample_y_train = np.asarray(to_categorical(sample_y_train))
sample_y_test = np.asarray(to_categorical(sample_y_test))

sample_data = shuffle(sample_data)  # For finding the learning rate, no validation set needed
sample_targets = np.asarray(to_categorical(sample_data['label']))

In [ ]:
def store_embeddings(vector_dir=vector_dir, vector_file=vector_file):
    embeddings = {}
    with open(os.path.join(vector_dir, vector_file)) as vectors:
        for line in vectors:
            values = line.split()
            word = values[0]
            weights = np.asarray(values[1:], dtype='float32')
            embeddings[word] = weights
    return embeddings
            
embeddings = store_embeddings()

In [ ]:
def create_embedding_matrix(max_vocab=max_vocab, embeddings=embeddings, word_index=word_index,
                            embedding_dim=embedding_dim):
    embedding_matrix = np.zeros((max_vocab + 1, embedding_dim)) # max_vocab + 1 to account for 0 as masking index
    for word, i in word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will remain all zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
            
embedding_matrix = create_embedding_matrix()

In [ ]:
# Construct attention layer

from tensorflow import matmul
class HierarchicalAttentionNetwork(Layer):
    ''''''
    def __init__(self, attention_dim):
        self.init_weights = initializers.get('glorot_normal')
        self.init_bias = initializers.get('zeros')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super().__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init_weights((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init_bias((self.attention_dim,)))
        self.u = K.variable(self.init_weights((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super().build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):        
        #uit = K.tile(K.expand_dims(self.W, axis=0), (K.shape(x)[0], 1, 1))
        #uit = matmul(x, uit)
        #uit = K.tanh(K.bias_add(uit, self.b))
        #ait = K.dot(uit, self.u)
        #ait = K.squeeze(ait, -1)

        #ait = K.exp(ait)
        
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.exp(K.squeeze(K.dot(uit, self.u), -1))
        
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        
        weighted_input = x * K.expand_dims(ait)
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
def build_model(attention_dim=attention_dim, GRU_dim=GRU_dim, drop=False, drop_pct=None,
                embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, word_index=word_index):
    
    embedding_layer = Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix],
                                input_length=max_words, trainable=False, mask_zero=True)

    #  Layers for processing words in each sentence with attention; output is encoded sentence vector 
    sentence_input = Input(shape=(max_words,), dtype='int32')
    embedded_sequences = embedding_layer(sentence_input)
    lstm_word = Bidirectional(GRU(GRU_dim, return_sequences=True))(embedded_sequences)
    attn_word = HierarchicalAttentionNetwork(attention_dim)(lstm_word)
    sentence_encoder = Model(sentence_input, attn_word)
    
    #  Layers for processing sentences in each article with attention; output is prediction
    article_input = Input(shape=(max_sentences, max_words), dtype='int32')
    article_encoder = TimeDistributed(sentence_encoder)(article_input)
    lstm_sentence = Bidirectional(GRU(GRU_dim, return_sequences=True))(article_encoder)
    attn_sentence = HierarchicalAttentionNetwork(attention_dim)(lstm_sentence)
    #  The Adam optimizer also will be tried and can take a dropout layer
    if drop:
        drop_sentence = Dropout(drop_pct)(attn_sentence)
        preds = Dense(2, activation='softmax')(drop_sentence)
    else:
        preds = Dense(2, activation='softmax')(attn_sentence)
    
    return Model(article_input, preds)

In [ ]:
# create checkpoints to save information from each training epoch

model_checkpoint = ModelCheckpoint(filepath=model_dir+'weights.{epoch:02d}.hdf5')
tb_checkpoint = TensorBoard(log_dir=tb_logs, histogram_freq=1, batch_size=128, write_graph=False, write_grads=True,
                            write_images=True)

if not os.path.exists(model_dir):
        os.makedirs(model_dir, exist_ok=True)
        
if not os.path.exists(tb_logs):
        os.makedirs(tb_logs, exist_ok=True)

The network will be trained using a cyclic learning rate as proposed in the 2017 paper [Cyclical Learning Rates for Training Neural Networks](https://arxiv.org/pdf/1506.01186.pdf). This enables quicker convergence by helping to avoid getting stuck in local minima or saddle points on the loss surface. A [reference implementation](https://github.com/bckenstler/CLR) is available on Github, and a three-post walk-through on [pyimagesearch](https://www.pyimagesearch.com/2019/07/29/cyclical-learning-rates-with-keras-and-deep-learning/) includes code in more detail. The approach has two steps. The first uses a learning rate vs. loss graph to determine the minimum and maximum learning rates. A Keras callback then varies the learning rate back and forth between these two values during training.

In [ ]:
start_lr = 1e-8
K.clear_session()
try:
    del model
except NameError:
    pass
model = build_model()
opt = SGD(lr=start_lr, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
from cyclic.rate_finder import LearningRateFinder
find_lr = LearningRateFinder(model)

find_lr_data = create_data_matrix(sample_data['content'])

In [ ]:
find_lr.find(find_lr_data, sample_targets, start_lr=1e-8, end_lr=1., epochs=3, batch_size=batch_size)

In [ ]:
find_lr.plot_loss()

The plot shows that effective learning (steady drop in loss) begins around $10^{-2.5}$, and
learning becomes erratic and the loss quickly explodes above $10^{-1}$. So the learning rates will cycle between these two values.

In [ ]:
from keras.callbacks import Callback
from cyclic.rate_cycler import CyclicLR

Using the min/max learning rates determined above, do test runs with the sample_train and sample_test data to determine the best choices for number of embedding, attention and GRU dimensions. Start with the smallest and see if increasing these dimensions results in significant performance improvement.

In [ ]:
clr = CyclicLR(epochs=4, num_samples=8000, batch_size=batch_size)

In [ ]:
K.clear_session()
try:
    del model
except NameError:
    pass
model = build_model()
opt = SGD(momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
model.fit(sample_train_matrix, sample_y_train, validation_data=(sample_test_matrix, sample_y_test),
          batch_size=batch_size, epochs=4, callbacks=[clr])

In [ ]:
clr.plot_lr()

In [ ]:
K.clear_session()
try:
    del model
except NameError:
    pass
model = build_model()
opt = SGD(momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
model.fit(sample_train_matrix, sample_y_train, validation_data=(sample_test_matrix, sample_y_test),
          batch_size=batch_size, epochs=4, callbacks=[clr])

Nesterov momentum did not improve performance, but the algorithms should also be tried without the cyclic learning rate, after which the Adam optimizer will be tried. A first experiment with Adam showed excessive overfitting, so it will be tried with a dropout layer.

In [ ]:
def run_trial(attention_dim=attention_dim, GRU_dim=GRU_dim, embedding_matrix=embedding_matrix, embedding_dim=embedding_dim,
              momentum=0.9, batch_size=batch_size, epochs=4, cyclic=False, nesterov=False, adam=False, drop_pct=0.5,
              X=sample_train_matrix, y=sample_y_train, X_val=sample_test_matrix, y_val=sample_y_test,
              word_index=word_index, scale_factor=0.5, base_lr=10**(-2.5), max_lr=1e-1):
    K.clear_session()
    try:
        del model
    except NameError:
        pass
    
    
    if adam is True:
        opt = Adam()
        model = build_model(attention_dim=attention_dim, GRU_dim=GRU_dim, embedding_matrix=embedding_matrix,
                            embedding_dim=embedding_dim, word_index=word_index, drop=True, drop_pct=drop_pct)
    else:
        opt = SGD(momentum=momentum, nesterov=nesterov)
        model = build_model(attention_dim=attention_dim, GRU_dim=GRU_dim, embedding_matrix=embedding_matrix,
                            embedding_dim=embedding_dim, word_index=word_index)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])
    
    if cyclic is True:
        clr = CyclicLR(epochs=epochs, num_samples=8000, batch_size=batch_size, scale_factor=scale_factor,
                       base_lr=base_lr, max_lr=max_lr)
        model.fit(X, y, validation_data=(sample_test_matrix, sample_y_test),
                  batch_size=batch_size, epochs=epochs, callbacks=[clr])
    else:
        model.fit(X, y, validation_data=(sample_test_matrix, sample_y_test),
                  batch_size=batch_size, epochs=epochs)

In [ ]:
run_trial()  # SGD without cyclic learning rate or Nesterov momentum

In [ ]:
run_trial(nesterov=True)  # SGD without cyclic learning rate but with Nesterov momentum

In [ ]:
run_trial(adam=True)  # Adam optimizer with default 50% dropout in feed to prediction layer

The 50% dropout somewhat lessened the gap between the training and validation accuracy scores, both of which are significantly higher than with SGD optimization, but the model is still overfitting significantly. Even worse, the validation accuracy has decreased during the last epoch, while the training accuracy crossed 90%, leaving not much room for improvement. Reducing the GRU and/or attention dimensions could help.

In [ ]:
run_trial(attention_dim=64, adam=True)

In [ ]:
run_trial(GRU_dim=32, attention_dim=64, adam=True)

In [ ]:
run_trial(GRU_dim=16, attention_dim=32, adam=True)

In [ ]:
run_trial(GRU_dim=16, attention_dim=16, adam=True, drop_pct=0.6)

In [ ]:
run_trial(GRU_dim=16, attention_dim=8, adam=True, drop_pct=0.6)

In [ ]:
run_trial(GRU_dim=8, attention_dim=8, adam=True, drop_pct=0.7)

Even with the GRU and attention dimensions as low as 8 and the dropout up to 70%, it can be seen that the training accuracy races ahead of the validation accuracy by the fourth epoch. Running it on the full dataset of 600,000+ articles would only make matters worse, although it is possible that good results could be obtained with a very few training epochs. The best performing combination for more extensive training without overfitting was the first, SGD with cyclic learning and no Nesterov momentum.

In [ ]:
run_trial(momentum=0.8, cyclic=True)  # Reduced momentum

Reducing momentum does not give any performance gain, so keep it at 0.9.

In [ ]:
run_trial(GRU_dim=100, attention_dim=200, cyclic=True)

Increasing the attention and GRU dimensions does not improve performance. Can these be reduced?

In [ ]:
run_trial(GRU_dim=50, attention_dim=100, cyclic=True)

In [ ]:
run_trial(GRU_dim=64, attention_dim=32, cyclic=True)

It appears that increasing the GRU dimension while decreasing the attention dimension improves performance. 

In [ ]:
run_trial(GRU_dim=128, attention_dim=32, cyclic=True)

Since the computation of the hidden state by the GRU seems the critival component, increasing the size of the word embedding and/or the vocabulary could improve performance. 

In [ ]:
embeddings_200 = store_embeddings(vector_file='glove.6B.200d.txt')
embedding_matrix_200 = create_embedding_matrix(embeddings=embeddings_200, embedding_dim=200)

In [ ]:
run_trial(GRU_dim=128, attention_dim=32, embedding_matrix=embedding_matrix_200, embedding_dim=200, cyclic=True)

In [ ]:
run_trial(GRU_dim=200, attention_dim=100, embedding_matrix=embedding_matrix_200, embedding_dim=200, cyclic=True)

In [ ]:
word_index_80k = {}
for ix, (word, _) in enumerate(words.most_common(80000)):
    word_index_80k[word] = ix + 1 # The zero index is reserved for masking out-of-vocab words

In [ ]:
sample_train_matrix_80k = create_data_matrix(sample_X_train, max_vocab=80000, word_index=word_index_80k)
sample_test_matrix_80k = create_data_matrix(sample_X_test, max_vocab=80000, word_index=word_index_80k)

embedding_matrix_100_80k = create_embedding_matrix(max_vocab=80000, word_index=word_index_80k)
embedding_matrix_200_80k = create_embedding_matrix(max_vocab=80000, embeddings=embeddings_200,
                                                   word_index=word_index_80k, embedding_dim=200)

In [ ]:
run_trial(GRU_dim=128, attention_dim=32, embedding_matrix=embedding_matrix_100_80k, cyclic=True,
          X=sample_train_matrix_80k, X_val=sample_test_matrix_80k, word_index=word_index_80k)

In [ ]:
run_trial(GRU_dim=200, attention_dim=100, embedding_matrix=embedding_matrix_100_80k, cyclic=True,
          X=sample_train_matrix_80k, X_val=sample_test_matrix_80k, word_index=word_index_80k)

In [ ]:
run_trial(GRU_dim=128, attention_dim=32, embedding_matrix=embedding_matrix_200_80k, embedding_dim=200, cyclic=True,
          X=sample_train_matrix_80k, X_val=sample_test_matrix_80k, word_index=word_index_80k)

In [ ]:
run_trial(GRU_dim=200, attention_dim=100, embedding_matrix=embedding_matrix_200_80k, embedding_dim=200, cyclic=True,
          X=sample_train_matrix_80k, X_val=sample_test_matrix_80k, word_index=word_index_80k)

In [ ]:
# Raise the base learning rate a little and increase the scale factor, so the max rate decreases more quickly
run_trial(GRU_dim=200, attention_dim=100, embedding_matrix=embedding_matrix_200_80k, embedding_dim=200, cyclic=True,
          X=sample_train_matrix_80k, X_val=sample_test_matrix_80k, word_index=word_index_80k, scale_factor=0.33,
          base_lr=1e-2)

In [ ]:
len(words)

In [ ]:
word_index_150k = {}
for ix, (word, _) in enumerate(words.most_common(150000)):
    word_index_150k[word] = ix + 1 # The zero index is reserved for masking out-of-vocab words
    
sample_train_matrix_150k = create_data_matrix(sample_X_train, max_vocab=150000, word_index=word_index_150k)
sample_test_matrix_150k = create_data_matrix(sample_X_test, max_vocab=150000, word_index=word_index_150k)

embedding_matrix_100_150k = create_embedding_matrix(max_vocab=150000, word_index=word_index_150k)
embedding_matrix_200_150k = create_embedding_matrix(max_vocab=150000, embeddings=embeddings_200,
                                                    word_index=word_index_150k, embedding_dim=200)

In [ ]:
run_trial(GRU_dim=128, attention_dim=32, embedding_matrix=embedding_matrix_100_150k, cyclic=True,
          X=sample_train_matrix_150k, X_val=sample_test_matrix_150k, word_index=word_index_150k)

In [ ]:
run_trial(GRU_dim=200, attention_dim=100, embedding_matrix=embedding_matrix_100_150k, cyclic=True,
          X=sample_train_matrix_150k, X_val=sample_test_matrix_150k, word_index=word_index_150k)

In [ ]:
run_trial(GRU_dim=128, attention_dim=32, embedding_matrix=embedding_matrix_200_150k, embedding_dim=200, cyclic=True,
          X=sample_train_matrix_150k, X_val=sample_test_matrix_150k, word_index=word_index_150k)

In [ ]:
run_trial(GRU_dim=200, attention_dim=100, embedding_matrix=embedding_matrix_200_150k, embedding_dim=200, cyclic=True,
          X=sample_train_matrix_150k, X_val=sample_test_matrix_150k, word_index=word_index_150k)

The 200-length vector embeddings performed better across the board. With the 150,000-word vocabulary, the network with GRU/attention dimensions of 128/32 performed almost as well as the 200/100 network, but the accuracy was increasing faster at the end of the four epochs and it took 3/5 of the training time. With the 80,000-word vocabulary, the 200/100 network performed better, after backtracking on the second epoch. Both of these will be tried with the full dataset. A minimum improvement of It is to be noted, however, that the accuracies obtained after four epochs are significantly lower than that obtained with tf-idf logistic regression. 

In [ ]:
len(bin(5))